## 递归之可变与不可变对象，具体见day77
#### python的不可变对象，如int，string，float，tuple ， 必须在每次递归时做return(visited, size = dfs(i, visited, size = size))，才能随着每次迭代进行改变
#### 而可变对象如：List Dict Set 则可以不用return（dfs(i, visited, size = size)）进行改变
#### 如果不进行赋值，最外层的 不可变对象 始终是递归初始的值，因此最终的 size 仍然是初次递归时 dfs(0) 中的 size。

In [ ]:
# 示例图，邻接表表示
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

def dfs(graph, start, visited=None):
    if not visited:
        visited = []
    visited.append(start)
    
    for i in graph[start]:
        if i not in visited:
            dfs(graph, i, visited)
    return visited

from collections import deque
def bfs(graph, start):
    queue = deque([start])
    visited = []

    while queue:
        q = queue.popleft()
        if q not in visited:
            visited.append(q)
            queue.extend(graph[q])
    
    return visited

dfs(graph, 'A'), bfs(graph, 'A')

In [2]:
# 构建邻接表
N = 4
dislikes = [[1,2],[1,3],[2,4]]

adj = {i: [] for i in range(1, N + 1)}
for a, b in dislikes:
    adj[a].append(b)
    adj[b].append(a)

adj

{1: [2, 3], 2: [1, 4], 3: [1], 4: [2]}

## 拓扑排序
#### （Topological Sorting）是一种用于有向无环图（DAG, Directed Acyclic Graph）节点排序的方法。拓扑排序广泛用于解决依赖关系的问题，如任务调度、编译依赖、课程安排等。

#### 给定一个 DAG，如果从 i 到 j 有边，则认为 j 依赖于 i。如果 i 到 j 有路径（i 可达 j），则称 j 间接依赖于 i。拓扑排序的目标是将所有节点排序，使得排在前面的节点不能依赖于排在后面的节点。

In [7]:
from collections import defaultdict

def topological_sort_dfs(N, edges):
    # 构建邻接表  有向图
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
    
    visited = [False] * N
    topo_order = []  # 存储拓扑排序结果
    temp_stack = set()  # 检测是否存在环

    def dfs(node):
        if node in temp_stack: 
            # 同一个node两次出现在temp_stack说明有环
            raise ValueError("Graph contains a cycle!")
        if visited[node]:
            return
        
        temp_stack.add(node)   
        visited[node] = True   
        
        for neighbor in graph[node]:
            dfs(neighbor)
        # 找到第一个没有依赖的节点加入topo_order开始往上回溯
        topo_order.append(node)  # 后序遍历   # 1 - 3 -2 - 0 - 5 - 4
        temp_stack.remove(node)  #  

    for node in range(N-1, -1, -1):
        if not visited[node]:
            dfs(node)
    
    return topo_order[::-1]  # 结果反转
    
# 示例图：N = 6, 节点从 0 到 5
edges = [(5, 2), (5, 0), (4, 0), (4, 1), (2, 3), (3, 1)]

try:
    result = topological_sort_dfs(6, edges)
    print("拓扑排序:", result)
except ValueError as e:
    print(e)

拓扑排序: [4, 5, 0, 2, 3, 1]


In [11]:
from collections import deque, defaultdict

def topological_sort_bfs(N, edges):
    # 构建邻接表
    graph = defaultdict(list)
    in_degree = defaultdict(int)  # 记录每个节点的入度
    
    # 构建图，并计算入度
    for u, v in edges:
        graph[u].append(v)
        in_degree[v] += 1
    
    # 将入度为 0 的节点加入队列
    queue = deque([node for node in range(N) if in_degree[node] == 0])
    topo_order = []
    
    while queue:
        node = queue.popleft()
        topo_order.append(node)
        
        # 对每个相邻节点，减少其入度，如果入度为 0，则加入队列
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # 如果拓扑排序包含的节点数少于 N，说明有环
    if len(topo_order) != N:
        raise ValueError("Graph contains a cycle!")
    
    return topo_order

# 示例图：N = 6, 节点从 0 到 5
edges = [(5, 2), (5, 0), (4, 0), (4, 1), (2, 3), (3, 1)]

try:
    result = topological_sort_bfs(6, edges)
    print("拓扑排序:", result)
except ValueError as e:
    print(e)


拓扑排序: [4, 5, 2, 0, 3, 1]


In [10]:
edges = [(5, 2), (5, 0), (4, 0), (4, 1), (2, 3), (3, 1)]
N = 6
graph = defaultdict(list)
in_degree = defaultdict(int) # 记录每个节点的入度

# 构建图，并计算入度
for u, v in edges:
    graph[u].append(v)
    in_degree[v] += 1

in_degree

defaultdict(int, {2: 1, 0: 2, 1: 2, 3: 1})

## 并查集
#### 合并（Union）：将两个元素所在的集合合并为一个集合。
#### 查找（Find）：查找某个元素所属的集合，通常返回这个集合的代表元素。
#### 并查集非常适合用于解决图的连通性问题，如判定图中某两个节点是否连通、动态维护连通分量等。

In [1]:
class UnionFind:
    def __init__(self, n):
        # 初始化 n 个元素，每个元素的父节点指向自己
        self.parent = [i for i in range(n)]
        # 初始化每个集合的大小（也可以理解为秩 rank，用于按秩合并）
        self.rank = [1] * n

    def find(self, x):
        # 查找 x 所在的集合根节点，并进行路径压缩
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        # 找到 x 和 y 的根节点
        rootX = self.find(x)
        rootY = self.find(y)
        
        # 如果两个元素属于不同的集合，则合并它们
        if rootX != rootY:
            # 按秩合并，将小树合并到大树上
            if self.rank[rootX] > self.rank[rootY]:
                self.parent[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.parent[rootX] = rootY
            else:
                # 如果 rank 相同，随便合并，并增加新树的 rank
                self.parent[rootY] = rootX
                self.rank[rootX] += 1

    def connected(self, x, y):
        # 判断 x 和 y 是否属于同一个集合
        return self.find(x) == self.find(y)


In [2]:
# 创建一个包含 5 个元素（0, 1, 2, 3, 4）的并查集
# uf = UnionFind(5)
# uf.parent, uf.rank

# 示例：连通性问题
n = 5  # 图中有5个节点，编号0到4
edges = [(0, 1), (1, 2), (3, 4)]  # 三条边

uf = UnionFind(n)

# 合并每条边
for u, v in edges:
    uf.union(u, v)

# 判断是否连通
print(uf.connected(0, 2))  # 输出：True
print(uf.connected(0, 3))  # 输出：False

True
False


In [5]:
uf.parent, uf.rank

([0, 0, 0, 3, 3], [2, 1, 1, 2, 1])